This implement is to extract entity and surrounding word and give it representative tag.
Say Entity E is made of words [e1, e2... ek]. Lets take -d and +d words as context. 

making e1-d......[e1, e2. .....ek]... ek+d as total entity words.

We would store doc_name, tag, entity and entity_context

Dictionary format

Element of Dictionary [ doc_id: dictonary of roles]

Key: Document names present in the corpus
Values: Dictionary of roles present in specific directory

Elements of Role Dictionary: 
Keys: Roles present in a specific doc
Value: List of entities having that role

Entity list itself is a list of words

In [1]:
import os, pickle

In [2]:
corpus_dir_train = '../../Data/input/train'
corpus_dir_test = '../../Data/input/test'
content_dir_name = 'content'
tag_dir_name = 'new_tags'

train_content_dir = os.path.join(corpus_dir_train, content_dir_name)
train_tag_dir = os.path.join(corpus_dir_train, tag_dir_name)
test_content_dir = os.path.join(corpus_dir_test, content_dir_name)
test_tag_dir = os.path.join(corpus_dir_test, tag_dir_name)

win_len = 10

In [3]:
ENTITY_ROLES = dict()

ENTITY_ROLES['PER_Victim'] = 0
ENTITY_ROLES['PER_Others'] = 0
ENTITY_ROLES['LOC_Event'] = 0
ENTITY_ROLES['ORG_Accused'] = 0
ENTITY_ROLES['ORG_Victim'] = 0
ENTITY_ROLES['LOC_Others'] = 0
ENTITY_ROLES['ORG_Others'] = 0
ENTITY_ROLES['PER_Accused'] = 0
ENTITY_ROLES['LOC_Accused'] = 0
ENTITY_ROLES['LOC_Victim'] = 0

In [4]:
def build_doc_role_entity_word_dictionary(content_dir, tag_dir, win_len):
    
    doc_role_entity_word_dict = dict()
    
    
    for f in os.listdir(content_dir):
        #print(f)
        
        role_entity_dict = dict()
        per_victim_list = list()
        per_accused_list = list()
        per_others_list = list()
        org_victim_list = list()
        org_accused_list = list()
        org_others_list = list()
        loc_victim_list = list()
        loc_accused_list = list()
        loc_others_list = list()
        loc_event_list = list()
        entity_doc_level_word_dict = dict()
    
        content_file = os.path.join(content_dir, f)
        tag_file = os.path.join(tag_dir, f)
        words = list()
        tags = list()
        with open(content_file, 'rt') as cf:
            for line in cf:
                for word in line.strip('\n').split():
                    words.append(word)

        with open(tag_file, 'rt') as tf:
            for line in tf:
                for tag in line.strip('\n').split():
                    tags.append(tag)
        word_len = len(words)
        tag_len = len(tags)
        if word_len == tag_len:
            i = 0
            while i < word_len:
        
                if tags[i] in ENTITY_ROLES.keys():
                    entity_start_index = i
                    entity_end_index = i 
                    while entity_end_index< word_len -1 and tags[entity_end_index] == tags[entity_end_index+1]:
                        entity_end_index = entity_end_index + 1
                    i = entity_end_index;
                    context_start_index = entity_start_index - win_len
                    context_end_index = entity_end_index + win_len
                    while context_start_index < 0:
                        context_start_index = context_start_index + 1
                    while context_end_index > word_len:
                        context_end_index = context_end_index - 1
                    entity_slice = slice(entity_start_index,entity_end_index+1)
                    context_slice = slice(context_start_index,context_end_index+1)
                    entity_words = '_'.join(words[entity_slice])
                    entity_tag = tags[entity_end_index]
                    sent_context_words = words[context_slice]
                    
                    if entity_words in entity_doc_level_word_dict.keys():
                        doc_context_words = entity_doc_level_word_dict[entity_words]
                        doc_context_words.extend(sent_context_words)
                        entity_doc_level_word_dict[entity_words] = doc_context_words
                       # print('Occured Again', '\n')
                    else:
                        entity_doc_level_word_dict[entity_words] = sent_context_words
                   # print(entity_words, entity_doc_level_word_dict[entity_words], '\n')
                    
                
                    if entity_tag == 'PER_Victim':
                        per_victim_list.append((entity_words, entity_doc_level_word_dict[entity_words]))
                
                    if entity_tag == 'PER_Others':
                        per_others_list.append((entity_words, entity_doc_level_word_dict[entity_words]))

                    if entity_tag == 'PER_Accused':
                        per_accused_list.append((entity_words, entity_doc_level_word_dict[entity_words]))

                    if entity_tag == 'LOC_Others':
                        loc_others_list.append((entity_words, entity_doc_level_word_dict[entity_words]))

                    if entity_tag == 'LOC_Event':
                        loc_event_list.append((entity_words, entity_doc_level_word_dict[entity_words]))

                    if entity_tag == 'LOC_Accused':
                        loc_accused_list.append((entity_words, entity_doc_level_word_dict[entity_words]))

                    if entity_tag == 'LOC_Victim':
                        loc_victim_list.append((entity_words, entity_doc_level_word_dict[entity_words]))

                    if entity_tag == 'ORG_Accused':
                        org_accused_list.append((entity_words, entity_doc_level_word_dict[entity_words]))

                    if entity_tag == 'ORG_Victim':
                        org_victim_list.append((entity_words, entity_doc_level_word_dict[entity_words]))

                    if entity_tag == 'ORG_Others':
                        org_others_list.append((entity_words, entity_doc_level_word_dict[entity_words]))
                
                    #print(words[entity_slice], tags[entity_end_index])
                    #print(words[context_slice])
                    #print('\n')
                i = i+1
            
        role_entity_dict['PER_Victim'] = per_victim_list
        role_entity_dict['PER_Others'] = per_others_list
        role_entity_dict['PER_Accused'] = per_accused_list
        role_entity_dict['LOC_Others'] = loc_others_list
        role_entity_dict['LOC_Event'] = loc_event_list
        role_entity_dict['LOC_Accused'] = loc_accused_list
        role_entity_dict['LOC_Victim'] = loc_victim_list
        role_entity_dict['ORG_Accused'] = org_accused_list
        role_entity_dict['ORG_Victim'] = org_victim_list
        role_entity_dict['ORG_Others'] = org_others_list
        
        doc_role_entity_word_dict[f] = role_entity_dict
    
    return doc_role_entity_word_dict

In [5]:
train_doc_entity_word_dict = build_doc_role_entity_word_dictionary(train_content_dir, train_tag_dir, win_len)
test_doc_entity_word_dict = build_doc_role_entity_word_dictionary(test_content_dir, test_tag_dir, win_len)

In [6]:
pickle.dump(train_doc_entity_word_dict, open('../../Data/output/EntityRep/train/doc_entity_doc_level_context_word.p', 'wb'))
pickle.dump(test_doc_entity_word_dict, open('../../Data/output/EntityRep/test/doc_entity_doc_level_context_word.p', 'wb'))